Instacart Online Retail Analysis
-----
This is an Integrated CA2 for CCT College Higher Diploma in Data Analytics for Business.
Courses: Machine Learning for Business / Data Visualization Techniques.
Main goals for this Analysis are:

-Build basic recommender systems (content based filtering and collaborative filtering).

-Apply Market Basket Analysis (Apriori and FP Growth).

-Design an interactive dashboard aimed at adults (+65 Years)

Libraries importation

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from mlxtend.frequent_patterns import apriori, association_rules

from ipywidgets import interact, widgets

Display and plot pre settings

In [2]:
pd.set_option('display.max_columns', 50)
pd.set_option('display.float_format', '{:.4f}'.format)

sns.set_theme(style="whitegrid")

plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['xtick.labelsize'] = 10
plt.rcParams['ytick.labelsize'] = 10
plt.rcParams['legend.fontsize'] = 10

Data Loading
---
In this section we are going to load four csv files and have a first look at their structure

In [3]:
orders = pd.read_csv('orders.csv')
order_products_prior = pd.read_csv('order_products__prior.csv')
products = pd.read_csv('products.csv')
departments = pd.read_csv('departments.csv')

orders.shape, order_products_prior.shape, products.shape , departments.shape

((3421083, 7), (32434489, 4), (49688, 4), (21, 2))

Now lets check the first rows of every dataset:

In [4]:
print("=== orders ===")
display(orders.head())

print("\n=== order_products_prior ===")
display(order_products_prior.head())

print("\n=== products ===")
display(products.head())

print("\n=== departments ===")
display(departments.head())


=== orders ===


,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0000
2,473747,1,prior,3,3,12,21.0000
3,2254736,1,prior,4,4,7,29.0000
4,431534,1,prior,5,4,15,28.0000



=== order_products_prior ===


,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0



=== products ===


,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13



=== departments ===


,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


Now lets see the info for each of them to have an overview of what are we going to clean and merge later on

In [5]:
print("=== orders.info() ===")
orders.info()
print("\n=== order_products_prior.info() ===")
order_products_prior.info()
print("\n=== products.info() ===")
products.info()
print("\n=== departments.info() ===")
departments.info()


=== orders.info() ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3421083 entries, 0 to 3421082
Data columns (total 7 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   order_id                int64  
 1   user_id                 int64  
 2   eval_set                object 
 3   order_number            int64  
 4   order_dow               int64  
 5   order_hour_of_day       int64  
 6   days_since_prior_order  float64
dtypes: float64(1), int64(5), object(1)
memory usage: 182.7+ MB

=== order_products_prior.info() ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32434489 entries, 0 to 32434488
Data columns (total 4 columns):
 #   Column             Dtype
---  ------             -----
 0   order_id           int64
 1   product_id         int64
 2   add_to_cart_order  int64
 3   reordered          int64
dtypes: int64(4)
memory usage: 989.8 MB

=== products.info() ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49688 entries, 0 to 496